<a href="https://colab.research.google.com/github/utensil/llm-playground/blob/main/notebooks/axolotl/runpod/axolotl-falcon-40b-qlora-deepspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning falcon-40b

- Axolotl+QLoRA
- minotaur datasets
- deepspeed ZeRO 3 8xGPU

<!-- https://jupyterlab.readthedocs.io/en/stable/user/commands.html#commands-list -->
<button data-commandLinker-command="apputils:change-theme" data-commandlinker-args='{"theme": "JupyterLab Dark"}' href="#">Dark theme</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/"}' href="#">llm-playground</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/"}' href="#">axolotl</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/notebooks/axolotl/runpod"}' href="#">Runpod notebooks</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/axolotl/examples"}' href="#">axolotl configs</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/storage"}' href="#">Storage</button>
<button data-commandLinker-command="filebrowser:go-to-path" data-commandlinker-args='{"path": "/workspace/llm-playground/axolotl-trained"}' href="#">axolotl-trained</button>
<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path": "/workspace/axolotl/examples/falcon/config-40b-qlora.yml"}' href="#">Edit config</button>

## Prepare

### Set HF Cache

In [ ]:
# %env HF_DATASETS_CACHE

In [ ]:
#%env TRANSFORMERS_CACHE

In [ ]:
!rm -rf /root/.cache

In [ ]:
!mkdir -p /content/cache

In [ ]:
!ln -s /content/cache /root/.cache

### HF Login

In [ ]:
# For axolotl push_dataset_to_hub
import os
from huggingface_hub import notebook_login, login
# Colab:
# notebook_login()
# RunPod:
login(os.environ.get("HUGGINGFACE_TOKEN"), add_to_git_credential=True)

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Update axolotl

In [ ]:
%cd /workspace/

/workspace


In [ ]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl axolotl-update

Cloning into 'axolotl-update'...
remote: Enumerating objects: 3148, done.
remote: Counting objects: 100% (1289/1289), done.
remote: Compressing objects: 100% (331/331), done.
remote: Total 3148 (delta 1043), reused 1082 (delta 907), pack-reused 1859
Receiving objects: 100% (3148/3148), 1.43 MiB | 3.29 MiB/s, done.
Resolving deltas: 100% (1979/1979), done.


In [ ]:
!cp -r axolotl-update/* axolotl

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   FAQS.md
	modified:   README.md
	modified:   docker/Dockerfile
	modified:   docker/Dockerfile-base
	modified:   examples/falcon/config-7b-lora.yml
	modified:   examples/falcon/config-7b.yml
	modified:   examples/gptq-lora-7b/README.md
	modified:   examples/gptq-lora-7b/config.yml
	modified:   examples/mpt-7b/config.yml
	modified:   requirements.txt
	modified:   scripts/finetune.py
	modified:   scripts/runpod-entrypoint.sh
	modified:   src/axolotl/datasets.py
	modified:   src/axolotl/flash_attn.py
	modified:   src/axolotl/prompt_strategies/alpaca_chat.py
	modified:   src/axolotl/prompters.py
	modified:   src/axolotl/utils/data.py
	modified:   src/axolotl/utils/models.py
	modified:   src/axolotl/utils/trainer.py
	modified:   src/axolotl/utils/validation.

In [ ]:
!pip install -e .

Obtaining file:///workspace/axolotl
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-install-c7qt2bvs/transformers_e0e367cf2eb84270a00604b3e969f20d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-install-c7qt2bvs/transformers_e0e367cf2eb84270a00604b3e969f20d
  Resolved https://github.com/huggingface/transformers.git to commit 70c79940957fb25b54bd1b106935c756b90345eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: axolotl
    Found existing installation: axolotl 0.1
    Uninstalling axolotl-0.1:
      Successfully uninstalled axolotl-0.1
  Running setup.py develop for axolotl


### Init Storage

In [ ]:
!python /workspace/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -m

Cloning https://huggingface.co/utensil/axolotl-trained into local empty directory.


In [ ]:
!ls /content/axolotl-trained

### Reinstall PyTorch with CUDA 11.8 (optional)

In [ ]:
!pip3 install -U torch --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 1.9 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 2.0.1+cu118 which is incompatible.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 2.0.1+cu118 which is incompatible.


### Reinstall deepspeed (optional)

In [ ]:
!ds_report

Setting ds_accelerator to cuda (auto detect)
--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
async_io ............... [YES] ...... [OKAY]
cpu_adagrad ............ [YES] ...... [OKAY]
cpu_adam ............... [YES] ...... [OKAY]
fused_adam ............. [YES] ...... [OKAY]
fused_lamb ............. [YES] ...... [OKAY]
quantizer .............. [YES] ...... [OKAY]
random_ltd ............. [YES] ...... [OKAY]
 [WARNING]  sparse_attn requires a torch vers

In [ ]:
!pip uninstall deepspeed

In [ ]:
!TORCH_CUDA_ARCH_LIST="3.5;5.0;6.0;6.1;7.0;7.5;8.0;8.6+PTX" DS_BUILD_OPS=1 pip install deepspeed --global-option="build_ext" --global-option="-j8" # --global-option="bdist_wheel"

### Init Configs

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
# Try no config
# !accelerate config default

Setting ds_accelerator to cuda (auto detect)
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
%%writefile ds_config.json
{
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    },
    "offload_param": {
      "device": "cpu",
      "pin_memory": true
    },
    "overlap_comm": true,
    "contiguous_gradients": true,
    "sub_group_size": 0,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 0,
    "stage3_max_reuse_distance": 0,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "bf16": {
    "enabled": "auto"
  },
  "fp16": {
    "enabled": "auto",
    "auto_cast": false,
    "loss_scale": 0,
    "initial_scale_power": 32,
    "loss_scale_window": 1000,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": "auto",
      "betas": [
        0.9,
        0.999
      ],
      "eps": 1e-8,
      "weight_decay": "auto"
    }
  },
  "train_batch_size": "auto",
  "gradient_accumulation_steps": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}

Overwriting ds_config.json


In [ ]:
%%writefile examples/falcon/config-40b-qlora.yml
# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: tiiuae/falcon-40b
base_model_config: tiiuae/falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all linear layers of the base model
lora_target_modules:
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project: falcon-qlora
wandb_watch:
wandb_run_id:
wandb_log_model:
output_dir: /content/axolotl-trained/falcon-qlora-40b-gsm8k/

# QLoRA paper Table 9
# - 16 for 7b & 13b
# - 32 for 33b, 64 for 64b
# Max size tested on A6000
# - 7b: 40
# - 40b: 4
# decrease if OOM, increase for max VRAM utilization
micro_batch_size: 1
gradient_accumulation_steps: 1
num_epochs: 3
# Optimizer for QLoRA
optimizer: paged_adamw_32bit
torchdistx_path:
lr_scheduler: cosine
# QLoRA paper Table 9
# - 2e-4 for 7b & 13b
# - 1e-4 for 33b & 64b
learning_rate: 0.0002
train_on_inputs: false
group_by_length: false
bf16: true
fp16: false
tf32: true
gradient_checkpointing: true
# stop training after this many evaluation losses have increased in a row
# https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer_callback.html#EarlyStoppingCallback
early_stopping_patience: 3
resume_from_checkpoint:
auto_resume_from_checkpoints: true
local_rank:
logging_steps: 1
xformers_attention: true
flash_attention:
gptq_groupsize:
gptq_model_v1:
warmup_steps: 10
eval_steps: 5
save_steps: 10
debug:
deepspeed:
weight_decay: 0.01
fsdp:
fsdp_config:
special_tokens:
  pad_token: "<|endoftext|>"
  bos_token: ">>ABSTRACT<<"
  eos_token: "<|endoftext|>"

Writing examples/falcon/config-40b-qlora.yml


In [ ]:
%%writefile scripts/ft.py
import os
from pathlib import Path
import fire
import logging
import finetune
from axolotl.utils.trainer import setup_trainer as setup_trainer_orig

logging.basicConfig(level=os.getenv("LOG_LEVEL", "INFO"))

def train_ex(
    config: Path = Path("configs/"),
    prepare_ds_only: bool = False,
    **kwargs,
):
  logging.info('train_ex before')
  finetune.train(config, prepare_ds_only, **kwargs)
  logging.info('train_ex after')

def setup_trainer_ex(cfg, train_dataset, eval_dataset, model, tokenizer):
  logging.info('setup_trainer_ex before')
  logging.info(f'cfg.some_config = {cfg.some_config}')
  trainer = setup_trainer_orig(cfg, train_dataset, eval_dataset, model, tokenizer)
  logging.info('setup_trainer_ex after')
  return trainer

finetune.setup_trainer = setup_trainer_ex

if __name__ == "__main__":
    fire.Fire(train_ex)

In [ ]:
%env ACCELERATE_USE_DEEPSPEED=true

env: ACCELERATE_USE_DEEPSPEED=true


# Training

## #1

In [ ]:
%cd /workspace/axolotl

/workspace/axolotl


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

# 1b: tiiuae/falcon-rw-1b
# 7b: tiiuae/falcon-7b
# 40b: tiiuae/falcon-40b
base_model: tiiuae/falcon-40b
base_model_config: tiiuae/falcon-40b
# required by falcon custom model code: https://huggingface.co/tiiuae/falcon-7b/tree/main
trust_remote_code: true
model_type: AutoModelForCausalLM
tokenizer_type: AutoTokenizer
load_in_8bit: false
# enable 4bit for QLoRA
load_in_4bit: true
gptq: false
strict: false

push_dataset_to_hub: utensil
hf_use_auth_token: true

datasets:
  - path: QingyiSi/Alpaca-CoT
    data_files:
      - Chain-of-Thought/formatted_cot_data/gsm8k_train.json
    type: "alpaca:chat"

dataset_prepared_path: last_run_prepared
val_set_size: 0.01
# enable QLoRA
adapter: qlora
lora_model_dir:
sequence_len: 2048
max_packed_sequence_len:

# hyperparameters from QLoRA paper Appendix B.2
# "We find hyperparameters to be largely robust across datasets"
lora_r: 64
lora_alpha: 16
# 0.1 for models up to 13B
# 0.05 for 33B and 65B models
lora_dropout: 0.05
# add LoRA modules on all line

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this

## #2


In [ ]:
%%writefile ds_config.json
{
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu",
      "pin_memory": true
    },
    "offload_param": {
      "device": "cpu",
      "pin_memory": true
    },
    "overlap_comm": true,
    "contiguous_gradients": true,
    "sub_group_size": 0,
    "reduce_bucket_size": "auto",
    "stage3_prefetch_bucket_size": "auto",
    "stage3_param_persistence_threshold": "auto",
    "stage3_max_live_parameters": 0,
    "stage3_max_reuse_distance": 0,
    "stage3_gather_16bit_weights_on_model_save": true
  },
  "bf16": {
    "enabled": "auto"
  },
  "fp16": {
    "enabled": "auto",
    "auto_cast": false,
    "loss_scale": 0,
    "initial_scale_power": 32,
    "loss_scale_window": 1000,
    "hysteresis": 2,
    "min_loss_scale": 1
  },
  "train_batch_size": "auto",
  "gradient_accumulation_steps": "auto",
  "train_micro_batch_size_per_gpu": "auto",
  "wall_clock_breakdown": false
}

Overwriting ds_config.json


In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this

## #3

After `ln -s`

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this

## #4

In [ ]:
!cat examples/falcon/config-40b-qlora.yml

In [ ]:
#%%writefile examples/falcon/config-40b-qlora.yml

In [ ]:
!accelerate launch scripts/finetune.py examples/falcon/config-40b-qlora.yml --deepspeed ds_config.json

Setting ds_accelerator to cuda (auto detect)
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `4`
		More than one GPU was found, enabling multi-GPU training.
		If this was unintended please pass in `--num_processes=1`.
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this

Deduplicated log:

```
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /root/miniconda3/envs/py3.9/lib/python3.9/site-packages/bitsandbytes-0.39.0-py3.9.egg/bitsandbytes/libbitsandbytes_cuda118.so...
Setting ds_accelerator to cuda (auto detect)
WARNING:root:`trust_remote_code` is set to true. Please make sure that you reviewed the remote code/model.
INFO:root:loading tokenizer... tiiuae/falcon-40b
Using bos_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using unk_token, but it is not set yet.
WARNING:datasets.builder:Found cached dataset parquet (/workspace/axolotl/''/utensil___parquet/utensil--31a4e867d804a957707db033c9abcd13-7b1208a23cdad6e9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 574.01it/s]
INFO:root:loading model and peft_config...
Loading checkpoint shards: 100%|██████████████████| 9/9 [01:31<00:00, 10.22s/it]
Loading checkpoint shards: 100%|██████████████████| 9/9 [01:32<00:00, 10.24s/it]
INFO:root:converting PEFT model w/ prepare_model_for_kbit_training
INFO:root:found linear modules: ['dense', 'dense_4h_to_h', 'dense_h_to_4h', 'query_key_value']
trainable params: 444334080 || all params: 21363310592 || trainable%: 2.0798933671188187
[2023-06-13 10:33:24,482] [WARNING] [comm.py:152:init_deepspeed_backend] NCCL backend in DeepSpeed not yet implemented
[2023-06-13 10:33:24,482] [INFO] [comm.py:594:init_distributed] cdb=None
[2023-06-13 10:33:24,482] [INFO] [comm.py:625:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
INFO:torch.distributed.distributed_c10d:Added key: store_based_barrier_key:1 to store for rank: 0
INFO:torch.distributed.distributed_c10d:Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
INFO:root:Compiling torch model
INFO:root:Pre-saving adapter config to /content/axolotl-trained/falcon-qlora-40b-gsm8k/
INFO:root:Starting trainer...
INFO:torch.distributed.distributed_c10d:Added key: store_based_barrier_key:2 to store for rank: 0
INFO:torch.distributed.distributed_c10d:Rank 0: Completed store-based barrier for key:store_based_barrier_key:2 with 4 nodes.
Parameter Offload: Total persistent parameters: 1982464 in 242 params
wandb: Currently logged in as: utensil. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.3
wandb: Run data is saved locally in /workspace/axolotl/wandb/run-20230613_103335-m0xv5jm1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run breezy-night-22
wandb: ⭐️ View project at https://wandb.ai/utensil/falcon-qlora
wandb: 🚀 View run at https://wandb.ai/utensil/falcon-qlora/runs/m0xv5jm1
  0%|                                                  | 0/5550 [00:00<?, ?it/s]
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Traceback (most recent call last):
  File "/workspace/axolotl/scripts/finetune.py", line 329, in <module>
    fire.Fire(train)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/fire/core.py", line 141, in Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/fire/core.py", line 475, in _Fire
    component = fn(*varargs, **kwargs)
  File "/workspace/axolotl/scripts/finetune.py", line 316, in train
    trainer.train(resume_from_checkpoint=resume_from_checkpoint)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/transformers/trainer.py", line 1537, in train
    return inner_training_loop(
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/transformers/trainer.py", line 1811, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/transformers/trainer.py", line 2643, in training_step
    self.accelerator.backward(loss)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/accelerator.py", line 1830, in backward
    self.deepspeed_engine_wrapped.backward(loss, **kwargs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/utils/deepspeed.py", line 167, in backward
    self.engine.backward(loss, **kwargs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/engine.py", line 1859, in backward
    self.optimizer.backward(loss, retain_graph=retain_graph)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/stage3.py", line 1962, in backward
    self.loss_scaler.backward(loss.float(), retain_graph=retain_graph)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/fp16/loss_scaler.py", line 63, in backward
    scaled_loss.backward(retain_graph=retain_graph)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/autograd/__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/autograd/function.py", line 274, in apply
    return user_fn(self, *args)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/utils/checkpoint.py", line 141, in backward
    outputs = ctx.run_function(*detached_inputs)
  File "/root/.cache/huggingface/modules/transformers_modules/tiiuae/falcon-40b/2ac60b04625e6694fb6143c00b9f93a01c7a000f/modelling_RW.py", line 642, in custom_forward
    return module(*inputs, use_cache=use_cache, output_attentions=output_attentions)
  File "/root/.cache/huggingface/modules/transformers_modules/tiiuae/falcon-40b/2ac60b04625e6694fb6143c00b9f93a01c7a000f/modelling_RW.py", line 396, in forward
    attn_outputs = self.self_attention(
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1538, in _call_impl
    result = forward_call(*args, **kwargs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/hooks.py", line 165, in new_forward
    output = old_forward(*args, **kwargs)
  File "/root/.cache/huggingface/modules/transformers_modules/tiiuae/falcon-40b/2ac60b04625e6694fb6143c00b9f93a01c7a000f/modelling_RW.py", line 252, in forward
    fused_qkv = self.query_key_value(hidden_states)  # [batch_size, seq_length, 3 x hidden_size]
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    result = hook(self, args)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/parameter_offload.py", line 371, in _pre_forward_module_hook
    self.pre_sub_module_forward_function(module)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/parameter_offload.py", line 483, in pre_sub_module_forward_function
    param_coordinator.fetch_sub_module(sub_module)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/partitioned_param_coordinator.py", line 251, in fetch_sub_module
    self.__all_gather_params(params_to_fetch)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/partitioned_param_coordinator.py", line 381, in __all_gather_params
    handle = partitioned_params[0].all_gather_coalesced(partitioned_params)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/utils/nvtx.py", line 15, in wrapped_fn
    ret_val = func(*args, **kwargs)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/zero/partition_parameters.py", line 928, in all_gather_coalesced
    dtype=get_only_unique_item(p.dtype for p in params),
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/deepspeed/runtime/utils.py", line 824, in get_only_unique_item
    raise RuntimeError(f"expected there to be only one unique element in {items}")
RuntimeError: expected there to be only one unique element in <generator object Init._convert_to_deepspeed_param.<locals>.all_gather_coalesced.<locals>.<genexpr> at 0x7f0f04211eb0>
wandb: Waiting for W&B process to finish... (failed 1). Press Control-C to abort syncing.
wandb: 🚀 View run breezy-night-22 at: https://wandb.ai/utensil/falcon-qlora/runs/m0xv5jm1
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230613_103335-m0xv5jm1/logs
WARNING:torch.distributed.elastic.multiprocessing.api:Sending process 8083 closing signal SIGTERM
ERROR:torch.distributed.elastic.multiprocessing.api:failed (exitcode: 1) local_rank: 1 (pid: 8084) of binary: /root/miniconda3/envs/py3.9/bin/python3
Traceback (most recent call last):
  File "/root/miniconda3/envs/py3.9/bin/accelerate", line 8, in <module>
    sys.exit(main())
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/commands/accelerate_cli.py", line 45, in main
    args.func(args)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/commands/launch.py", line 939, in launch_command
    multi_gpu_launcher(args)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/accelerate/commands/launch.py", line 628, in multi_gpu_launcher
    distrib_run.run(args)
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/distributed/run.py", line 785, in run
    elastic_launch(
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/distributed/launcher/api.py", line 134, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
  File "/root/miniconda3/envs/py3.9/lib/python3.9/site-packages/torch/distributed/launcher/api.py", line 250, in launch_agent
    raise ChildFailedError(
torch.distributed.elastic.multiprocessing.errors.ChildFailedError:
============================================================
scripts/finetune.py FAILED
------------------------------------------------------------
Failures:
Root Cause (first observed failure):
[0]:
  time      : 2023-06-13_10:33:45
  host      : 5523bc3f4c77
  rank      : 1 (local_rank: 1)
  exitcode  : 1 (pid: 8084)
  error_file: <N/A>
  traceback : To enable traceback see: https://pytorch.org/docs/stable/elastic/errors.html
============================================================
```

# Upload

### Upload checkpoints to HF

In [ ]:
%cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/

In [ ]:
!ls -lhta |grep checkpoint-

In [ ]:
!ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}'

In [ ]:
# ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf

In [ ]:
!python /workspace/llm-playground/helper/storage.py utensil/axolotl-trained /content/ -u

## Below are ad hoc cells handling issues during training

current out dir:

```
/content/axolotl-trained/falcon-qlora-40b-gsm8k/
```

### Force release VRAM

In [ ]:
# First interupt the kernel, wait a few seconds then run this to kill finetune to release VRAM
!ps aux|grep python|grep finetune|awk '{print $2}'|xargs kill

### Clean the finetuned model and all checkpoints

In [ ]:
# Only run this to start over
!rm -rf /content/axolotl-trained/falcon-qlora-40b-gsm8k/

### Zip the prepared dataset

In [ ]:
!apt install zip
!zip -r last_run_prepared.zip -xi last_run_prepared

### Monitoring GPU

In [ ]:
# Run this in a seperate terminal
!nvitop -m full

### Fix DISK FULL

In [ ]:
%cd /

/


In [ ]:
!du -d 2 -h|grep G

In [ ]:
!du -d 2 -h /root/.local

In [ ]:
!rm -rf /root/.local/share/Trash/

In [ ]:
!rm -rf /root/.local/share/wandb/

In [ ]:
!rm -rf /root/.cache/wandb/

### Check who is using GPU

In [ ]:
!apt install lsof

In [ ]:
!lsof /dev/nvidia*

### A new bash without tmux etc.

In [ ]:
!bash --norc --noprofile

### Clean up all checkpoints but last one

In [ ]:
cd /content/axolotl-trained/falcon-qlora-40b-gsm8k/ && ls -lhta |grep checkpoint- | awk 'NR > 1 {print $9}' | xargs rm -rf